# Recommendation system

Improvement of the recommendation system from ".\recommender.ipynb"

In [1]:
import pandas as pd
import numpy as np

In [2]:
r_cols = ['user_id','movie_id','rating']
ratings = pd.read_csv('.\\MLCourse\\ml-100k\\u.data', sep = "\t", names = r_cols, usecols=range(3))

m_cols = ['movie_id','title']
movies = pd.read_csv('.\\MLCourse\\ml-100k\\u.item', sep = "|", names = m_cols, usecols=range(2), encoding='latin-1')

ratings = pd.merge(movies,ratings)
ratings.shape


(100003, 4)

In [3]:
movieRatings = ratings.pivot_table(index=['user_id'], columns=['title'], values = 'rating')
movieRatings

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 1st idea

Re-scaling rates

 {1:-2, 2:-1, 3:0, 4:1, 5:2}

In [5]:
movieRatings.columns

"'Til There Was You (1997)"

In [23]:
[i**2 for i in [-2,-1,0,1,2]]

[4, 1, 0, 1, 4]

In [24]:
[i**2 for i in [1,2,3,4,5]]

[1, 4, 9, 16, 25]

In [82]:
scale =  {1:-2, 2:-1, 3:0, 4:1, 5:2}
movieRatingsScaled = movieRatings.replace(scale)
movieRatingsScaled.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,-1.0,2.0,NaN,NaN,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN,1.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
corrMatrix = movieRatingsScaled.corr(method='pearson', min_periods=100) #filtering for movies rated at least 100 times
corrMatrix

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-900 (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187 (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Young Guns II (1990),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Young Poisoner's Handbook, The (1995)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zeus and Roxanne (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
movieRatings[["Empire Strikes Back, The (1980)",'Cinderella (1950)']].dropna()

title,"Empire Strikes Back, The (1980)",Cinderella (1950)
user_id,,
5,5.0,3.0
7,4.0,4.0
13,5.0,2.0
16,5.0,5.0
18,3.0,3.0
...,...,...
892,5.0,4.0
897,4.0,4.0
912,3.0,4.0


In [84]:
corrMatrix["Empire Strikes Back, The (1980)"].sort_values(ascending=False).head(10)

title
Empire Strikes Back, The (1980)              1.000000
Star Wars (1977)                             0.748353
Return of the Jedi (1983)                    0.721229
Raiders of the Lost Ark (1981)               0.538659
Bridge on the River Kwai, The (1957)         0.356743
Cinderella (1950)                            0.349920
Back to the Future (1985)                    0.345285
Terminator 2: Judgment Day (1991)            0.333532
Field of Dreams (1989)                       0.328415
Indiana Jones and the Last Crusade (1989)    0.313236
Name: Empire Strikes Back, The (1980), dtype: float64

In [85]:
corrMatrix["Gone with the Wind (1939)"].sort_values(ascending=False).head(10)

title
Gone with the Wind (1939)            1.000000
Wizard of Oz, The (1939)             0.430219
E.T. the Extra-Terrestrial (1982)    0.361463
Schindler's List (1993)              0.344765
Graduate, The (1967)                 0.326215
Amadeus (1984)                       0.323631
M*A*S*H (1970)                       0.313956
It's a Wonderful Life (1946)         0.305688
Casablanca (1942)                    0.291137
Back to the Future (1985)            0.257319
Name: Gone with the Wind (1939), dtype: float64

In [86]:
corrMatrix["Star Wars (1977)"].sort_values(ascending=False).head(10)

title
Star Wars (1977)                                      1.000000
Empire Strikes Back, The (1980)                       0.748353
Return of the Jedi (1983)                             0.672556
Raiders of the Lost Ark (1981)                        0.536117
Austin Powers: International Man of Mystery (1997)    0.377433
Sting, The (1973)                                     0.367538
Indiana Jones and the Last Crusade (1989)             0.350107
Frighteners, The (1996)                               0.332729
L.A. Confidential (1997)                              0.319065
Dumbo (1941)                                          0.317656
Name: Star Wars (1977), dtype: float64

In [27]:
movieRatings.loc[0].dropna()

title
Empire Strikes Back, The (1980)    5.0
Gone with the Wind (1939)          1.0
Star Wars (1977)                   5.0
Name: 0, dtype: float64

In [87]:
movieRatingsScaled.loc[0].dropna()

title
Empire Strikes Back, The (1980)    2.0
Gone with the Wind (1939)         -2.0
Star Wars (1977)                   2.0
Name: 0, dtype: float64

In [88]:
myRatings = movieRatingsScaled.loc[0].dropna() #user 0
similarCandidates = pd.Series()

for i in range(0, len(myRatings.index)):
    print("Adding similarities for "+myRatings.index[i]+"...")

    #retrieve similar movies to this on that I rated
    sims = corrMatrix[myRatings.index[i]].dropna()

    #now scale its similarity by how well I rated this movie
    sims = sims.map(lambda x: x*myRatings[i])

    #add the score to the list of similarity candidates

    similarCandidates = similarCandidates._append(sims)

    similarCandidates = similarCandidates.drop(myRatings.index)

    similarCandidates = similarCandidates.drop_duplicates()

    similarCandidates = similarCandidates.groupby(similarCandidates.index).sum()

print("sorting...")
similarCandidates.sort_values(inplace=True, ascending=False)
print(similarCandidates.head(10))

Adding similarities for Empire Strikes Back, The (1980)...
Adding similarities for Gone with the Wind (1939)...
Adding similarities for Star Wars (1977)...
sorting...
Return of the Jedi (1983)               2.369077
Raiders of the Lost Ark (1981)          1.857918
Bridge on the River Kwai, The (1957)    1.346646
Cinderella (1950)                       1.298165
Dumbo (1941)                            1.192658
Star Trek: The Wrath of Khan (1982)     1.187232
Batman (1989)                           1.179026
Princess Bride, The (1987)              1.117470
Star Trek: Generations (1994)           1.113909
Firm, The (1993)                        1.083683
dtype: float64


C:\Users\ville\AppData\Local\Temp\ipykernel_20140\2242663362.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sims = sims.map(lambda x: x*myRatings[i])
C:\Users\ville\AppData\Local\Temp\ipykernel_20140\2242663362.py:15: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  similarCandidates = similarCandidates._append(sims)
C:\Users\ville\AppData\Local\Temp\ipykernel_20140\2242663362.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by 

The instructor's technique: correlating the raw ratings then multiplying by the rating and summing.

The instructor's results: (top 10)

- Empire Strikes Back, The (1980)              **8.877450**

- Star Wars (1977)                             **8.870971**

- Return of the Jedi (1983)                    **7.178172**

- Raiders of the Lost Ark (1981)               **5.519700**

- Indiana Jones and the Last Crusade (1989)    **3.488028**

- Bridge on the River Kwai, The (1957)         **3.366616**

- Back to the Future (1985)                    **3.357941**

- Sting, The (1973)                            **3.329843**

- Cinderella (1950)                            **3.245412**

- Field of Dreams (1989)                       **3.222311**



## Result

I believe that the result with scale + instructor's technique was worse.

My reference is Cinderella should be lower on the list or should not show up.

# 2nd Idea

Re-scaling rates

 {1:-4, 2:-1, 3:0, 4:1, 5:4}

In [89]:
scale =  {1:-4, 2:-1, 3:0, 4:1, 5:4}
movieRatingsScaled = movieRatings.replace(scale)
movieRatingsScaled.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,-1.0,4.0,NaN,NaN,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,4.0,0.0,NaN,NaN,NaN,1.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
corrMatrix = movieRatingsScaled.corr(method='pearson', min_periods=100) #filtering for movies rated at least 100 times
corrMatrix.head(10)

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-900 (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187 (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2 Days in the Valley (1996),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"20,000 Leagues Under the Sea (1954)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001: A Space Odyssey (1968),NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,0.02604,NaN,NaN,NaN,NaN,NaN,NaN
3 Ninjas: High Noon At Mega Mountain (1998),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
myRatings = movieRatingsScaled.loc[0].dropna() #user 0
similarCandidates = pd.Series()

for i in range(0, len(myRatings.index)):
    print("Adding similarities for "+myRatings.index[i]+"...")

    #retrieve similar movies to this on that I rated
    sims = corrMatrix[myRatings.index[i]].dropna()

    #now scale its similarity by how well I rated this movie
    sims = sims.map(lambda x: x*myRatings[i])

    #add the score to the list of similarity candidates

    similarCandidates = similarCandidates._append(sims)

    similarCandidates = similarCandidates.drop(myRatings.index)

    similarCandidates = similarCandidates.drop_duplicates()

    similarCandidates = similarCandidates.groupby(similarCandidates.index).max()

print("sorting...")
similarCandidates.sort_values(inplace=True, ascending=False)
print(similarCandidates.head(10))

Adding similarities for Empire Strikes Back, The (1980)...
Adding similarities for Gone with the Wind (1939)...
Adding similarities for Star Wars (1977)...
sorting...
Return of the Jedi (1983)                    2.774228
Raiders of the Lost Ark (1981)               2.006105
Cinderella (1950)                            1.471336
Indiana Jones and the Last Crusade (1989)    1.441546
Back to the Future (1985)                    1.376345
E.T. the Extra-Terrestrial (1982)            1.346736
Sting, The (1973)                            1.342819
Chasing Amy (1997)                           1.332645
Frighteners, The (1996)                      1.314759
Terminator 2: Judgment Day (1991)            1.264900
dtype: float64


C:\Users\ville\AppData\Local\Temp\ipykernel_20140\296583248.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sims = sims.map(lambda x: x*myRatings[i])
C:\Users\ville\AppData\Local\Temp\ipykernel_20140\296583248.py:15: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  similarCandidates = similarCandidates._append(sims)
C:\Users\ville\AppData\Local\Temp\ipykernel_20140\296583248.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by pos

In [93]:
corrMatrix["Empire Strikes Back, The (1980)"].sort_values(ascending=False).head(10)

title
Empire Strikes Back, The (1980)              1.000000
Return of the Jedi (1983)                    0.693557
Star Wars (1977)                             0.671968
Raiders of the Lost Ark (1981)               0.470435
Cinderella (1950)                            0.367834
Back to the Future (1985)                    0.344086
Chasing Amy (1997)                           0.333161
Terminator 2: Judgment Day (1991)            0.316225
Star Trek: The Wrath of Khan (1982)          0.310223
Indiana Jones and the Last Crusade (1989)    0.299391
Name: Empire Strikes Back, The (1980), dtype: float64

## Result

I believe that the result with new scale + instructor's technique was worse.

My reference is Cinderella should be lower on the list or should not show up.

# 3rd Idea

Normalize rates

In [122]:
[float(i)/sum([1,2,3,4,5]) for i in [1,2,3,4,5]]

[0.06666666666666667,
 0.13333333333333333,
 0.2,
 0.26666666666666666,
 0.3333333333333333]

In [137]:
scale = {}
for i in [1,2,3,4,5]:
    scaled_value = float(i)/sum([1,2,3,4,5]) 
    scale[i] = scaled_value
movieRatingsScaled = movieRatings.replace(scale)
movieRatingsScaled.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.133333,0.333333,NaN,NaN,0.2,0.266667,NaN,NaN,...,NaN,NaN,NaN,0.333333,0.2,NaN,NaN,NaN,0.266667,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.066667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.133333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
corrMatrix = movieRatingsScaled.corr(method='pearson', min_periods=100) #filtering for movies rated at least 100 times
corrMatrix.head(10)

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-900 (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187 (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2 Days in the Valley (1996),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"20,000 Leagues Under the Sea (1954)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001: A Space Odyssey (1968),NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,-0.001307,NaN,NaN,NaN,NaN,NaN,NaN
3 Ninjas: High Noon At Mega Mountain (1998),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
myRatings = movieRatingsScaled.loc[0].dropna() #user 0
similarCandidates = pd.Series()

for i in range(0, len(myRatings.index)):
    print("Adding similarities for "+myRatings.index[i]+"...")

    #retrieve similar movies to this on that I rated
    sims = corrMatrix[myRatings.index[i]].dropna()

    #now scale its similarity by how well I rated this movie
    sims = sims.map(lambda x: x*myRatings[i])

    #add the score to the list of similarity candidates

    similarCandidates = similarCandidates._append(sims)

    similarCandidates = similarCandidates.drop(myRatings.index)

    similarCandidates = similarCandidates.drop_duplicates()

    similarCandidates = similarCandidates.groupby(similarCandidates.index).sum()

print("sorting...")
similarCandidates.sort_values(inplace=True, ascending=False)
print(similarCandidates.head(10))

Adding similarities for Empire Strikes Back, The (1980)...
Adding similarities for Gone with the Wind (1939)...
Adding similarities for Star Wars (1977)...
sorting...
Return of the Jedi (1983)                    0.478545
Raiders of the Lost Ark (1981)               0.367980
Indiana Jones and the Last Crusade (1989)    0.232535
Bridge on the River Kwai, The (1957)         0.224441
Back to the Future (1985)                    0.223863
Sting, The (1973)                            0.221990
Cinderella (1950)                            0.216361
Field of Dreams (1989)                       0.214821
Wizard of Oz, The (1939)                     0.213351
Dumbo (1941)                                 0.198776
dtype: float64


C:\Users\ville\AppData\Local\Temp\ipykernel_20140\2242663362.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sims = sims.map(lambda x: x*myRatings[i])
C:\Users\ville\AppData\Local\Temp\ipykernel_20140\2242663362.py:15: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  similarCandidates = similarCandidates._append(sims)
C:\Users\ville\AppData\Local\Temp\ipykernel_20140\2242663362.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by 

In [124]:
similarCandidates.head(15)

Return of the Jedi (1983)                    0.478545
Raiders of the Lost Ark (1981)               0.367980
Indiana Jones and the Last Crusade (1989)    0.232535
Bridge on the River Kwai, The (1957)         0.224441
Back to the Future (1985)                    0.223863
Sting, The (1973)                            0.221990
Cinderella (1950)                            0.216361
Field of Dreams (1989)                       0.214821
Wizard of Oz, The (1939)                     0.213351
Dumbo (1941)                                 0.198776
E.T. the Extra-Terrestrial (1982)            0.198150
Star Trek: The Wrath of Khan (1982)          0.197872
Batman (1989)                                0.196504
Jaws (1975)                                  0.195788
Casablanca (1942)                            0.192231
dtype: float64

The instructor's technique: correlating the raw ratings then multiplying by the rating and summing.

The instructor's results: (top 10)

- Empire Strikes Back, The (1980)              **8.877450**

- Star Wars (1977)                             **8.870971**

- Return of the Jedi (1983)                    **7.178172**

- Raiders of the Lost Ark (1981)               **5.519700**

- Indiana Jones and the Last Crusade (1989)    **3.488028**

- Bridge on the River Kwai, The (1957)         **3.366616**

- Back to the Future (1985)                    **3.357941**

- Sting, The (1973)                            **3.329843**

- Cinderella (1950)                            **3.245412**

- Field of Dreams (1989)                       **3.222311**



# 4th Idea

Scaling and normalising at the user level

In [140]:
movieRatings.head(10)

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN
6,NaN,NaN,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,4.0,NaN,NaN,5.0,5.0,NaN,4.0,...,NaN,NaN,NaN,5.0,3.0,NaN,3.0,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
movieRatingsScaled = movieRatings.div(movieRatings.sum(axis=1), axis=0)
movieRatingsScaled.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.002047,0.005118,NaN,NaN,0.003071,0.004094,NaN,NaN,...,NaN,NaN,NaN,0.005118,0.003071,NaN,NaN,NaN,0.004094,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004425,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.013605,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
corrMatrix = movieRatingsScaled.corr(method='pearson', min_periods=100) #filtering for movies rated at least 100 times
corrMatrix.head(10)

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-900 (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187 (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2 Days in the Valley (1996),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"20,000 Leagues Under the Sea (1954)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001: A Space Odyssey (1968),NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,0.865908,NaN,NaN,NaN,NaN,NaN,NaN
3 Ninjas: High Noon At Mega Mountain (1998),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
myRatings = movieRatingsScaled.loc[0].dropna() #user 0
similarCandidates = pd.Series()

for i in range(0, len(myRatings.index)):
    print("Adding similarities for "+myRatings.index[i]+"...")

    #retrieve similar movies to this on that I rated
    sims = corrMatrix[myRatings.index[i]].dropna()

    #now scale its similarity by how well I rated this movie
    sims = sims.map(lambda x: x*myRatings[i])

    #add the score to the list of similarity candidates

    similarCandidates = similarCandidates._append(sims)

    similarCandidates = similarCandidates.drop(myRatings.index)

    similarCandidates = similarCandidates.drop_duplicates()

    similarCandidates = similarCandidates.groupby(similarCandidates.index).max()

print("sorting...")
similarCandidates.sort_values(inplace=True, ascending=False)
print(similarCandidates.head(10))

Adding similarities for Empire Strikes Back, The (1980)...
Adding similarities for Gone with the Wind (1939)...
Adding similarities for Star Wars (1977)...
sorting...
Return of the Jedi (1983)                    0.445324
12 Angry Men (1957)                          0.438550
Raiders of the Lost Ark (1981)               0.434776
Abyss, The (1989)                            0.431544
Tomorrow Never Dies (1997)                   0.430605
Wrong Trousers, The (1993)                   0.430335
Rumble in the Bronx (1995)                   0.429758
Hoop Dreams (1994)                           0.429549
Indiana Jones and the Last Crusade (1989)    0.428366
Good Will Hunting (1997)                     0.428207
dtype: float64


C:\Users\ville\AppData\Local\Temp\ipykernel_20140\296583248.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sims = sims.map(lambda x: x*myRatings[i])
C:\Users\ville\AppData\Local\Temp\ipykernel_20140\296583248.py:15: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  similarCandidates = similarCandidates._append(sims)
C:\Users\ville\AppData\Local\Temp\ipykernel_20140\296583248.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by pos

In [151]:
similarCandidates.head(20)

Return of the Jedi (1983)                    0.445324
12 Angry Men (1957)                          0.438550
Raiders of the Lost Ark (1981)               0.434776
Abyss, The (1989)                            0.431544
Tomorrow Never Dies (1997)                   0.430605
Wrong Trousers, The (1993)                   0.430335
Rumble in the Bronx (1995)                   0.429758
Hoop Dreams (1994)                           0.429549
Indiana Jones and the Last Crusade (1989)    0.428366
Good Will Hunting (1997)                     0.428207
Toy Story (1995)                             0.426448
Vertigo (1958)                               0.425952
North by Northwest (1959)                    0.425932
Amadeus (1984)                               0.425842
Gandhi (1982)                                0.425667
Butch Cassidy and the Sundance Kid (1969)    0.425208
E.T. the Extra-Terrestrial (1982)            0.424469
Boot, Das (1981)                             0.423744
Much Ado About Nothing (1993

In [150]:
similarCandidates['Cinderella (1950)']

0.42363234321273535

## Best result!